In [1]:
import json, pprint, requests, textwrap
host = 'http://localhost:8998'
data = {'kind': 'spark'}
headers = {'Content-Type': 'application/json'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{'id': 0,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'spark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [2]:
session_url = host + r.headers['location']
r = requests.get(session_url, headers=headers)
r.json()

{'id': 0,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'spark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [3]:
statements_url = session_url + '/statements'
data = {'code': '1 + 1'}
r = requests.post(statements_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 0,
 'code': '1 + 1',
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

In [4]:
statement_url = host + r.headers['location']
r = requests.get(statement_url, headers=headers)
pprint.pprint(r.json())

{'code': '1 + 1',
 'completed': 1681605210311,
 'id': 0,
 'output': {'data': {'text/plain': 'res0: Int = 2\n'},
            'execution_count': 0,
            'status': 'ok'},
 'progress': 1.0,
 'started': 1681605207219,
 'state': 'available'}


In [5]:
data = {'kind': 'pyspark'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{'id': 1,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [6]:
data = {
  'code': textwrap.dedent("""
    import random
    NUM_SAMPLES = 100000
    def sample(p):
      x, y = random.random(), random.random()
      return 1 if x*x + y*y < 1 else 0

    count = sc.parallelize(xrange(0, NUM_SAMPLES)).map(sample).reduce(lambda a, b: a + b)
    print "Pi is roughly %f" % (4.0 * count / NUM_SAMPLES)
    """)
}

r = requests.post(statements_url, data=json.dumps(data), headers=headers)
pprint.pprint(r.json())

{'code': '\n'
         'import random\n'
         'NUM_SAMPLES = 100000\n'
         'def sample(p):\n'
         '  x, y = random.random(), random.random()\n'
         '  return 1 if x*x + y*y < 1 else 0\n'
         '\n'
         'count = sc.parallelize(xrange(0, '
         'NUM_SAMPLES)).map(sample).reduce(lambda a, b: a + b)\n'
         'print "Pi is roughly %f" % (4.0 * count / NUM_SAMPLES)\n',
 'completed': 0,
 'id': 1,
 'output': None,
 'progress': 0.0,
 'started': 0,
 'state': 'waiting'}


In [7]:
statement_url = host + r.headers['location']
r = requests.get(statement_url, headers=headers)
pprint.pprint(r.json())

{'code': '\n'
         'import random\n'
         'NUM_SAMPLES = 100000\n'
         'def sample(p):\n'
         '  x, y = random.random(), random.random()\n'
         '  return 1 if x*x + y*y < 1 else 0\n'
         '\n'
         'count = sc.parallelize(xrange(0, '
         'NUM_SAMPLES)).map(sample).reduce(lambda a, b: a + b)\n'
         'print "Pi is roughly %f" % (4.0 * count / NUM_SAMPLES)\n',
 'completed': 1681605592046,
 'id': 1,
 'output': {'ename': 'Error',
            'evalue': "<console>:2: error: '.' expected but ';' found.",
            'execution_count': 1,
            'status': 'error',
            'traceback': ['NUM_SAMPLES = 100000\n', '^\n']},
 'progress': 1.0,
 'started': 1681605591908,
 'state': 'available'}
